# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Lian  ->  J. Lian  |  ['J. Lian']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 57 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2306.13792


extracting tarball to tmp_2306.13792...

 done.
Retrieving document from  https://arxiv.org/e-print/2306.14100


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2306.13792/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2306.14100... done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:730: LatexWarning: Could not extract abstract from tmp_2306.14100/main-NA.tex
  warnings.warn(LatexWarning(f"Could not extract abstract from {self.main_file}"))


list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.14100-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.14100) | **Unusual integrated metallicity profile of our Milky Way**  |
|| <mark>J. Lian</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-06-27*|
|*Comments*| *34 pages, 6 figures, published online in Nature Astronomy with open access on 22 June 2023. This is the version prior to the peer review. The published version is available here: this https URL*|
|**Abstract**| The heavy element abundance profiles in galaxies place stringent constraint on galaxy growth and assembly history. Low-redshift galaxies generally have a negative metallicity gradient in their gas and stars. Such gradients are thought to be a natural manifestation of galaxy inside-out formation. As the Milky Way is currently the only spiral galaxy in which we can measure temporally-resolved chemical abundances, it enables unique insights into the origin of metallicity gradients and their correlation with the growth history of galaxies. However, until now, these unique abundance profiles had not been translated into the integrated-light measurements needed to seamlessly compare with the general galaxy population. Here we report the first measurement of the light-weighted, integrated stellar metallicity profile of our Galaxy. We find that the integrated metallicity profile of the Milky Way has a '$\wedge$'-shape broken metallicity profile, with a mildly positive gradient inside a Galactocentric radius of 7 kpc and a steep negative gradient outside. This metallicity profile appears unusual when compared to Milky Way-mass star-forming galaxies observed in the MaNGA survey and simulated in the TNG50 cosmological simulation. The analysis of the TNG50 simulated galaxies suggests that the Milky Way's positive inner gradient may be due to an inside-out quenching process. The steep negative gradient in the outer disc, however, is challenging to explain in the simulations. Our results suggest the Milky Way may not be a typical spiral galaxy for its mass regarding metallicity distribution and thus offers insight into the variety of galaxy enrichment processes. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.13792-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.13792) | **Maximizing science return by coordinating the survey strategies of Roman  with Rubin, and other major facilities**  |
|| R. Street, et al. |
|*Appeared on*| *2023-06-27*|
|*Comments*| *Submitted to the 2023 call for White Papers regarding the Roman Mission Core Community Surveys*|
|**Abstract**| [Abridged] The Nancy Grace Roman Space Telescope will be one of several flagship survey facilities operating over the next decade starting $\sim$2025. The deep near-IR imaging that Roman will deliver will be highly complementary to the capabilities of other survey telescopes that will operate contemporaneously, particularly those that can provide data at different wavelengths and messengers, or different time intervals. Combining data from multiple facilities can provide important astrophysical insights, provided the data acquisition is carefully scheduled, and careful plans are made for appropriate joint data analyses. In this White Paper, we discuss the broad range of science that would be enabled by coordinating Roman observations of the Galactic Bulge with those of the Vera C. Rubin Observatory. Specifically, we discuss how Roman's characterization of lensing events caused by exoplanets, stellar systems and stellar remnants can be enhanced by data from Rubin. The same data will also be highly advantageous for the determination of stellar properties, and for distinguishing exoplanetary transits. It will enable more accurate period-color-luminosity relationships to be measured for RR~Lyrae throughout the Milky Way Bulge and Bar, probing galactic structure and dynamics. But we stress that this is only a sample of the full potential and advocate for a more complete study to be made as a joint effort between these major projects. We note that we do not suggest any changes beyond the established Science Requirements for the RGBTDS, in terms of survey footprint or filter selection. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2306.14100.md
    + _build/html/tmp_2306.14100/./feh-prof-ages-4Gyr.png
    + _build/html/tmp_2306.14100/./z-prof-comp-kpc-gas.png
    + _build/html/tmp_2306.14100/./grad-in-out-gas-hist.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\doi}[1]{\url{https://doi.org/#1}}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\araa}{ARA\&A}$
$\newcommand{\aap}{A\&A}$
$\newcommand{\aj}{AJ}$
$\newcommand{\apjl}{ApJL}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\ao}{ApOpt}$
$\newcommand{\mb}[1]{\textcolor{blue}{#1}}$
$\newcommand{\jl}[1]{\textcolor{purple}{#1}}$
$\newcommand{\rr}[1]{\textcolor{green}{#1}}$
$\newcommand{\ap}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\mbnew}[1]{\textcolor{orange}{#1}}$
$\newcommand{\ha}{\hbox{H\alpha}}$
$\newcommand{\hb}{\hbox{H\beta}}$
$\newcommand{\oii}{\hbox{[O {\sc ii}]}}$
$\newcommand{\oiii}{\hbox{[O {\sc iii}]}}$
$\newcommand{\hg}{\hbox{H\gamma}}$
$\newcommand{\nii}{\hbox{[N {\sc ii}]}}$
$\newcommand{\}{url}$
$\newcommand{\urlprefix}{URL }$</div>



<div id="title">

# Unusual integrated metallicity profile of our Milky Way

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2306.14100-b31b1b.svg)](https://arxiv.org/abs/2306.14100)<mark>Appeared on: 2023-06-27</mark> -  _34 pages, 6 figures, published online in Nature Astronomy with open access on 22 June 2023. This is the version prior to the peer review. The published version is available here: this https URL_

</div>
<div id="authors">

<mark>J. Lian</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** The heavy element abundance profiles in galaxies place stringent constraint on galaxy growth and assembly history. Low-redshift galaxies generally have a negative metallicity gradient in their gas and stars. Such gradients are thought to be a natural manifestation of galaxy inside-out formation. As the Milky Way is currently the only spiral galaxy in which we can measure temporally-resolved chemical abundances, it enables unique insights into the origin of metallicity gradients and their correlation with the growth history of galaxies. However, until now, these unique abundance profiles had not been translated into the integrated-light measurements needed to seamlessly compare with the general galaxy population. Here we report the first measurement of the light-weighted, integrated stellar metallicity profile of our Galaxy. We find that the integrated metallicity profile of the Milky Way has a '$\wedge$'-shape broken metallicity profile, with a mildly positive gradient inside a Galactocentric radius of 7 kpc and a steep negative gradient outside. This metallicity profile appears unusual when compared to Milky Way-mass star-forming galaxies observed in the MaNGA survey and simulated in the TNG50 cosmological simulation. The analysis of the TNG50 simulated galaxies suggests that the Milky Way's positive inner gradient may be due to an inside-out quenching process. The steep negative gradient in the outer disc, however, is challenging to explain in the simulations. Our results suggest the Milky Way may not be a typical spiral galaxy for its mass regarding metallicity distribution and thus offers insight into the variety of galaxy enrichment processes. 

</div>

<div id="div_fig1">

<img src="tmp_2306.14100/./feh-prof-ages-4Gyr.png" alt="Fig1" width="100%"/>

**Figure 1. -** Average light-weighted stellar iron and magnesium abundance profiles of the Milky Way galaxy as a whole and of three different age populations.
	The integrated metallicity of all ages and that in each age bin are average values of mono-abundance populations, weighted by their optical r-band luminosity (Methods). The size of the colourful squares indicates the fraction of the total luminosity at each radial bin contained in each mono-age component.
	 (*z-prof-age*)

</div>
<div id="div_fig2">

<img src="tmp_2306.14100/./z-prof-comp-kpc-gas.png" alt="Fig2" width="100%"/>

**Figure 2. -** {\sl Left:} Average radial light-weighted stellar metallicity profile of the Milky Way (black) in comparison with those of  low-redshift Milky Way-mass star-forming galaxies in the MaNGA survey (blue) and in the TNG50 simulation (magenta), averaged across 544 and 134 galaxies, respectively.
	Black errorbars indicate the stochastic uncertainty of the integrated stellar metallicity of the Milky Way.
	The blue and magenta shaded regions represent the 1 $\sigma$ scatter of the distributions of these Milky Way-mass star forming galaxies in the MaNGA survey and TNG50 simulation, respectively. The filled symbols and errorbars denote their median metallicity profile and the error of the median. {\sl Right:} Comparison among the same galaxy samples as on the left but now for their present-day metallicity gradients. For MaNGA galaxies, we use the oxygen abundance ([O/H]) measured in their ionized gas from optical emission lines (Methods) to represent their present-day metallicity. For the Milky Way and TNG50 galaxies, we use [Mg/H], which is tightly correlated with [O/H]\citep{ting2022}, of their young (0-4 Gyr) population to represent their present-day metallicity (Methods).
	 (*z-prof*)

</div>
<div id="div_fig3">

<img src="tmp_2306.14100/./grad-in-out-gas-hist.png" alt="Fig3" width="100%"/>

**Figure 3. -** Comparison of the light-weighted average stellar metallicity gradients in- and outside the break radius ($\rm R_{b}$ -- panels a and b) and of the present-day gradient (panel c) of the Milky Way with those of Milky Way-mass star-forming galaxies in the MaNGA survey and the TNG50 simulation. In panels a and b, the distributions of MaNGA and TNG50 galaxies are colour coded by their number in each pixel. In both cases, the Milky Way is located at the bottom right edge of the distributions of the observed or simulated galaxy populations. In panel c, the shaded region indicates the 1 $\sigma$ stochastic uncertainty of the Milky Way's present-day gradient. For reference, we include the gradient of oxygen or magnesium abundance of different types of young objects in the Milky Way, including OB stars \citep{braganca2019}, Cepheids \citep{genovali2015}, open cluster \citep{magrini2017}, and H II regions \citep{wenger2019}.
	 (*zgrad-hist2d*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2306.14100"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

221  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

1  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
